In [88]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

# 반환값이 컬럼객체다
from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType

In [2]:
sc = SparkContext( 'local' ) # spark://ip:7070 -> 분산 클러스터 이용가능 
sqlCtx = SQLContext( sc )

In [5]:
df = sqlCtx.read.csv( path   = '../data/crime_in_Seoul.csv',
                      header = True,
                      encoding = 'euc-kr',
                      inferSchema = True # shcema 자동 추정,
                    )
# thousands option이 없음 -> 수동으로 , 제거해서 숫자로 

df.show(4)

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|  종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
only showing top 4 rows



In [6]:
df.printSchema()

root
 |-- 관서명: string (nullable = true)
 |-- 살인 발생: integer (nullable = true)
 |-- 살인 검거: integer (nullable = true)
 |-- 강도 발생: integer (nullable = true)
 |-- 강도 검거: integer (nullable = true)
 |-- 강간 발생: integer (nullable = true)
 |-- 강간 검거: integer (nullable = true)
 |-- 절도 발생: string (nullable = true)
 |-- 절도 검거: string (nullable = true)
 |-- 폭력 발생: string (nullable = true)
 |-- 폭력 검거: string (nullable = true)



In [9]:
type(df)

pyspark.sql.dataframe.DataFrame

## 분산데이터 프레임에서 head사용하면
- > collect()된 row별로 나온다

In [12]:
df.head(5) # row별로 도출

[Row(관서명='중부서', 살인 발생=2, 살인 검거=2, 강도 발생=3, 강도 검거=2, 강간 발생=105, 강간 검거=65, 절도 발생='1,395', 절도 검거='477', 폭력 발생='1,355', 폭력 검거='1,170'),
 Row(관서명='종로서', 살인 발생=3, 살인 검거=3, 강도 발생=6, 강도 검거=5, 강간 발생=115, 강간 검거=98, 절도 발생='1,070', 절도 검거='413', 폭력 발생='1,278', 폭력 검거='1,070'),
 Row(관서명='남대문서', 살인 발생=1, 살인 검거=0, 강도 발생=6, 강도 검거=4, 강간 발생=65, 강간 검거=46, 절도 발생='1,153', 절도 검거='382', 폭력 발생='869', 폭력 검거='794'),
 Row(관서명='서대문서', 살인 발생=2, 살인 검거=2, 강도 발생=5, 강도 검거=4, 강간 발생=154, 강간 검거=124, 절도 발생='1,812', 절도 검거='738', 폭력 발생='2,056', 폭력 검거='1,711'),
 Row(관서명='혜화서', 살인 발생=3, 살인 검거=2, 강도 발생=5, 강도 검거=4, 강간 발생=96, 강간 검거=63, 절도 발생='1,114', 절도 검거='424', 폭력 발생='1,015', 폭력 검거='861')]

# Row

In [15]:
r = Row( aa=10, bb=20, cc=30 )
print(type(r))
r

<class 'pyspark.sql.types.Row'>


Row(aa=10, bb=20, cc=30)

## 딕셔너리처럼 인덱싱 가능하다

In [28]:
r['aa']

10

In [27]:
for i in df.head(5):
    print(i['관서명'], i['살인 발생'])

중부서 2
종로서 3
남대문서 1
서대문서 2
혜화서 3


In [29]:
# 전체 row의 개수
df.count()

31

In [30]:
# 전체 row 보기
df.show( df.count() )

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|  종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|    2,056|    1,711|
|  혜화서|        3|        2|        5|        4|       96|       63|    1,114|      424|    1,015|      861|
|  용산서|        5|        5|       14|       14|      194|      173|    1,557|      587|    2,050|    1,704|
|  성북서|        2|        2|        2|        1|       86|       

In [33]:
df.describe().toPandas() # 결과값이 분산 데이터 프레임

,summary,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,count,31,31,31,31,31,31,31,31,31,31,31
1,mean,None,5.258064516129032,4.935483870967742,8.903225806451612,8.161290322580646,175.7741935483871,131.96774193548387,814.25,638.1428571428571,708.0,746.5
2,stddev,None,3.6052530372174423,3.203492717550306,4.9352800576372395,5.710205916856522,85.58025850136987,56.59003673849767,123.27577485729573,224.73963771863822,227.6883835420683,172.61614447476612
3,min,강남서,1,0,1,1,59,46,"1,063","1,129","1,015","1,015"
4,max,혜화서,14,12,22,26,334,247,953,984,869,861


## 특정 컬럼 선택하기

In [39]:
df.select( '관서명','강도 발생' ).show(5)

+--------+---------+
|  관서명|강도 발생|
+--------+---------+
|  중부서|        3|
|  종로서|        6|
|남대문서|        6|
|서대문서|        5|
|  혜화서|        5|
+--------+---------+
only showing top 5 rows



In [47]:
df.orderBy( '강도 발생',ascending=0 ).select('관서명','강도 발생').show(5)
# 0 = False / 1 = True

+--------+---------+
|  관서명|강도 발생|
+--------+---------+
|영등포서|       22|
|  구로서|       15|
|  강남서|       15|
|  용산서|       14|
|  마포서|       14|
+--------+---------+
only showing top 5 rows



In [50]:
df.select( '관서명','강도 발생' ).orderBy( '강도 발생', ascending=0 ).show(5)

+--------+---------+
|  관서명|강도 발생|
+--------+---------+
|영등포서|       22|
|  구로서|       15|
|  강남서|       15|
|  용산서|       14|
|  마포서|       14|
+--------+---------+
only showing top 5 rows



## 조건 설정하기

In [58]:
df.where('`강도 발생` >= 5' ).select('관서명','강도 발생').orderBy('강도 발생').show(5)

+--------+---------+
|  관서명|강도 발생|
+--------+---------+
|  혜화서|        5|
|서대문서|        5|
|  종로서|        6|
|  수서서|        6|
|남대문서|        6|
+--------+---------+
only showing top 5 rows



## 문제
- 살인 발생이 5건 이상인 관서명과 살인 발생을 추출하시오

In [65]:
df.select('관서명','살인 발생').\
    where( '`살인 발생` >= 5' ).\
    orderBy( '살인 발생', ascending=0 ).\
    show()

+--------+---------+
|  관서명|살인 발생|
+--------+---------+
|영등포서|       14|
|  중랑서|       13|
|  송파서|       11|
|  수서서|       10|
|  노원서|       10|
|  관악서|        9|
|  구로서|        8|
|  마포서|        8|
|  강서서|        7|
|  서초서|        7|
|  강북서|        7|
|  용산서|        5|
|동대문서|        5|
|  동작서|        5|
+--------+---------+



In [71]:
df.where( "`관서명` == '마포서' or  `관서명` == '동대문서' " ).show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|    2,548|    2,227|
|  마포서|        8|        8|       14|       10|      294|      247|    2,555|      813|    2,983|    2,519|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



In [73]:
df.where( " `관서명` in ('중부서','종로서') ").show()

+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|    1,355|    1,170|
|종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|    1,278|    1,070|
+------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+



# 변수를 사용해야하는 경우

In [77]:
a = 2
b = 7
df.where( f" `살인 발생` >= {a} and `살인 발생` <= {b}").select('관서명','살인 발생').show()

+--------+---------+
|  관서명|살인 발생|
+--------+---------+
|  중부서|        2|
|  종로서|        3|
|서대문서|        2|
|  혜화서|        3|
|  용산서|        5|
|  성북서|        2|
|동대문서|        5|
|  성동서|        4|
|  동작서|        5|
|  광진서|        4|
|  서부서|        2|
|  강북서|        7|
|  금천서|        3|
|  강남서|        3|
|  강서서|        7|
|  강동서|        4|
|  종암서|        3|
|  서초서|        7|
|  양천서|        3|
|  도봉서|        3|
+--------+---------+



In [78]:
a = 2
b = 7
df.where( f" `살인 발생` between {a} and {b}").select('관서명','살인 발생').show()

+--------+---------+
|  관서명|살인 발생|
+--------+---------+
|  중부서|        2|
|  종로서|        3|
|서대문서|        2|
|  혜화서|        3|
|  용산서|        5|
|  성북서|        2|
|동대문서|        5|
|  성동서|        4|
|  동작서|        5|
|  광진서|        4|
|  서부서|        2|
|  강북서|        7|
|  금천서|        3|
|  강남서|        3|
|  강서서|        7|
|  강동서|        4|
|  종암서|        3|
|  서초서|        7|
|  양천서|        3|
|  도봉서|        3|
+--------+---------+



In [80]:
df.drop( '폭력 발생','폭력 검거' ).show()

+--------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|    1,395|      477|
|  종로서|        3|        3|        6|        5|      115|       98|    1,070|      413|
|남대문서|        1|        0|        6|        4|       65|       46|    1,153|      382|
|서대문서|        2|        2|        5|        4|      154|      124|    1,812|      738|
|  혜화서|        3|        2|        5|        4|       96|       63|    1,114|      424|
|  용산서|        5|        5|       14|       14|      194|      173|    1,557|      587|
|  성북서|        2|        2|        2|        1|       86|       71|      953|      409|
|동대문서|        5|        5|       13|       13|      173|      146|    1,981|      814|
|  마포서|        8|        8|       14|       10|      294|    

In [82]:
c = df['폭력 검거'] # pandas는 폭력 검거 column에 대한 **Series**가 나옴
type(c)  # Spark DataFrame은 **Column 객체**

pyspark.sql.column.Column

# withColumn( 컬럼명, 컬럼 객체 )
- test라는 컬럼이 있으면 **수정**
- 없으면 **추가**

In [86]:
df.withColumn( 'test', df['살인 검거']*2  ).toPandas()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,test
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",4
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",6
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794,0
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",4
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861,4
5,용산서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",10
6,성북서,2,2,2,1,86,71,953,409,"1,194","1,015",4
7,동대문서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",10
8,마포서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",16
9,영등포서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",24


In [90]:
c = col('강도 검거') # 컬럼 객체
type(c)

pyspark.sql.column.Column

In [95]:
df['강도 검거'] # 컬럼 객체

Column<b'\xea\xb0\x95\xeb\x8f\x84 \xea\xb2\x80\xea\xb1\xb0'>

In [91]:
df.withColumn( '정도', col('강도 검거') ).toPandas()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,정도
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",2
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",5
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794,4
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",4
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861,4
5,용산서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",14
6,성북서,2,2,2,1,86,71,953,409,"1,194","1,015",1
7,동대문서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",13
8,마포서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",10
9,영등포서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",20


In [118]:
df.withColumn( '정도', df['강도 검거'] ).toPandas()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,정도
0,중부서,2,2,3,2,105,65,1395,477,"1,355","1,170",2
1,종로서,3,3,6,5,115,98,1070,413,"1,278","1,070",5
2,남대문서,1,0,6,4,65,46,1153,382,869,794,4
3,서대문서,2,2,5,4,154,124,1812,738,"2,056","1,711",4
4,혜화서,3,2,5,4,96,63,1114,424,"1,015",861,4
5,용산서,5,5,14,14,194,173,1557,587,"2,050","1,704",14
6,성북서,2,2,2,1,86,71,953,409,"1,194","1,015",1
7,동대문서,5,5,13,13,173,146,1981,814,"2,548","2,227",13
8,마포서,8,8,14,10,294,247,2555,813,"2,983","2,519",10
9,영등포서,14,12,22,20,295,183,2964,978,"3,572","2,961",20


## 조건 부여 가능

In [94]:
df.withColumn( '정도',   when( df['강도 검거'] >= 10, '많음' ).\
                         when( df['강도 검거'] >= 5, '보통' ).\
                         otherwise( '적음' ) 
             ).toPandas()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,정도
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",적음
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",보통
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794,적음
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",적음
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861,적음
5,용산서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",많음
6,성북서,2,2,2,1,86,71,953,409,"1,194","1,015",적음
7,동대문서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",많음
8,마포서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",많음
9,영등포서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",많음


In [104]:
ff = udf( lambda x: '많음' if x >= 10 else '적음')
df.withColumn( '정도1', ff(df['강도 검거']) ).toPandas()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,정도1
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",적음
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",적음
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794,적음
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",적음
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861,적음
5,용산서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",많음
6,성북서,2,2,2,1,86,71,953,409,"1,194","1,015",적음
7,동대문서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",많음
8,마포서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",많음
9,영등포서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",많음


## 문자열 숫자형으로 변환

In [108]:
# 절도 발생 컬럼의 ,를 제거
df.withColumn( '절도 발생', regexp_replace( '절도 발생', ',', '')).toPandas()[['관서명','절도 발생']]

,관서명,절도 발생
0,중부서,1395
1,종로서,1070
2,남대문서,1153
3,서대문서,1812
4,혜화서,1114
5,용산서,1557
6,성북서,953
7,동대문서,1981
8,마포서,2555
9,영등포서,2964


In [109]:
df = df.withColumn( '절도 발생', regexp_replace( '절도 발생', ',', ''))
df.show(5)

+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  관서명|살인 발생|살인 검거|강도 발생|강도 검거|강간 발생|강간 검거|절도 발생|절도 검거|폭력 발생|폭력 검거|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  중부서|        2|        2|        3|        2|      105|       65|     1395|      477|    1,355|    1,170|
|  종로서|        3|        3|        6|        5|      115|       98|     1070|      413|    1,278|    1,070|
|남대문서|        1|        0|        6|        4|       65|       46|     1153|      382|      869|      794|
|서대문서|        2|        2|        5|        4|      154|      124|     1812|      738|    2,056|    1,711|
|  혜화서|        3|        2|        5|        4|       96|       63|     1114|      424|    1,015|      861|
+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
only showing top 5 rows



In [110]:
df.printSchema() # 절도 발생은 여전히 string이다.-> 타입 변경 해줘야함.

root
 |-- 관서명: string (nullable = true)
 |-- 살인 발생: integer (nullable = true)
 |-- 살인 검거: integer (nullable = true)
 |-- 강도 발생: integer (nullable = true)
 |-- 강도 검거: integer (nullable = true)
 |-- 강간 발생: integer (nullable = true)
 |-- 강간 검거: integer (nullable = true)
 |-- 절도 발생: string (nullable = true)
 |-- 절도 검거: string (nullable = true)
 |-- 폭력 발생: string (nullable = true)
 |-- 폭력 검거: string (nullable = true)



In [117]:
df = df.withColumn( '절도 발생', df['절도 발생'].cast( IntegerType() ) )
df.printSchema()

root
 |-- 관서명: string (nullable = true)
 |-- 살인 발생: integer (nullable = true)
 |-- 살인 검거: integer (nullable = true)
 |-- 강도 발생: integer (nullable = true)
 |-- 강도 검거: integer (nullable = true)
 |-- 강간 발생: integer (nullable = true)
 |-- 강간 검거: integer (nullable = true)
 |-- 절도 발생: integer (nullable = true)
 |-- 절도 검거: string (nullable = true)
 |-- 폭력 발생: string (nullable = true)
 |-- 폭력 검거: string (nullable = true)



In [ ]:
}